# Linked Art - Visualisation - John Ruskin

This notebook creates a timeline visualisation of artworks by John Ruskin, from Linked Art JSON-LD.


## Import What We Need

See the [Transform Introduction](01-00-Transform-Intro.ipynb) for a description of the Python libraries used

In [53]:
import os

try:
    import json
except:
    !pip install json
    import json 
     
try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

In [54]:
outputdir = "./data/ruskin/output/json"


In [55]:
file_list=os.listdir(r"./data/ruskin/output/json/")
  
sources = {}
sources["https://www.nga.gov/collection/"] = {"name":"National Gallery of Art","colour": "#E8A798"}
sources["https://clevelandart.org/art/"] = {"name":"Cleveland Museum of Art", "colour":"#EDD59E"}
sources["https://www.philamuseum.org/collection/object/"] = {"name":"Philadelphia Museum of Art","colour":"#6B5876"}
sources["https://www.tate.org.uk/art/artworks/"] = {"name":"Tate Museum","colour":"#A09998"}
sources["https://www.harvardartmuseums.org/collections/object/"] = {"name":"Harvard Art Museum","colour":"#00758F"}
sources["ashmolean"] = {"name":"Ashmolean Museum","colour":"#3f83e8"}
sources["rijk"] = {"name":"Rijksmuseum","colour":"#94b7c3"}

    
json_vis = {"events":[]} 
json_vis["title"] = {
    "media": {
        "url": "//upload.wikimedia.org/wikipedia/commons/0/0a/John_Ruskin_1863.jpg",
        "caption": "John Ruskin",
        "credit": "Wikipedia"
        } ,
        "text": {
        "headline": "John Ruskin",
        "text": "<p>This timeline visualisation shows artworks created by John Ruskin.</p><p>It demonstrates how the Linked Art data model can be used to transform, reconcile and visualise collections data for artworks.</p> <p>See https://linked.art for more information.</p>"
        }  
}
json_vis["eras"] = [{
        "start_date": {
            "year": "1819" 
        },
        "end_date":{
            "year": "1900"
        },
        "text": {"headline":"John Ruskin's lifetime"
                }
    }]

for file in file_list:
    # read file and append to 
    with open( "data/ruskin/output/json/" + file) as json_file:
        artwork = json.load(json_file)
        
        if "_label" not in artwork:
            continue
        if "produced_by" not in artwork:
            continue
        if "begin_of_the_begin" not in artwork["produced_by"]["timespan"]:
            continue
            
        if "end_of_the_end" in artwork["produced_by"]["timespan"]:
            
            if "begin_of_the_begin" in artwork["produced_by"]["timespan"] and abs(int(artwork["produced_by"]["timespan"]["end_of_the_end"]) - int(artwork["produced_by"]["timespan"]["begin_of_the_begin"])) > 10:
                continue
            
            
        if artwork["produced_by"]["timespan"]["begin_of_the_begin"] in  (1819,""):
            continue
    
        id = artwork["id"]
        credit = ""
        text = ""
        imageurl = ""
        bgcolour = ""
        homepage = ""
        
        for source in list(sources.keys()):
            if source in id:
                credit = sources[source]["name"]
                bgcolour = sources[source]["colour"]
            
        if "referred_to_by" in artwork and len(artwork["referred_to_by"]) > 0 and "content" in artwork["referred_to_by"][0]:  
            text = artwork["referred_to_by"][0]["content"]
      
    
        if artwork["subject_of"][0]["classified_as"][1]["id"] == "http://vocab.getty.edu/aat/300266277":
            homepage = artwork["subject_of"][0]["id"]
            text = text + "<br/><br/><a target='_new' href='" + homepage+ "'>Artwork homepage</a>"
    
        if artwork["representation"][0]["id"] != "":
            imageurl = artwork["representation"][0]["id"]
            if imageurl == "" or "placeholder" in imageurl:
                continue
 
        
        if "begin_of_the_begin" in artwork["produced_by"]["timespan"]:
            begin = artwork["produced_by"]["timespan"]["begin_of_the_begin"]
        try:
            begin = int(begin)
        except:
            begin = ""
    
        if "end_of_the_end" in artwork["produced_by"]["timespan"]:
            end = artwork["produced_by"]["timespan"]["end_of_the_end"]
        try:
            end = int(end)
        except:
            end = ""
            
        json_vis["events"].append({
            "media": {
            "url": imageurl,
            "caption": "",
            "credit": credit,
            "thumbnail": imageurl
            },
            "display_date": artwork["produced_by"]["timespan"]["_label"],
            "start_date": {
                "year": begin },
            
            "end_date": {
                "year": end
            },
            "text": {
            "headline": artwork["_label"],
            "text":  text
            } ,
            "type": "title",
            "background":{"color":bgcolour}
        })
    
text_file = open('./data/ruskin/output/vis.json', 'wt')  
n = text_file.write(json.dumps(json_vis,indent=2))
text_file.close()
        

## Timeline Visualisation

Follow instructions at https://timeline.knightlab.com/#make to make a timeline

In [56]:
%%HTML
<!-- 1 -->
        <link title="timeline-styles" rel="stylesheet" 
              href="https://cdn.knightlab.com/libs/timeline3/latest/css/timeline.css">

        <!-- 2 -->
        <script src="https://cdn.knightlab.com/libs/timeline3/latest/js/timeline.js"></script>

        <div id='timeline-embed' style="width: 100%; height: 800px"></div>

        <!-- 3 -->
        <script type="text/javascript">   
            window.timeline = new TL.Timeline('timeline-embed', "data/ruskin/output/vis.json");
        </script>